## Yahoo Fundamental

In [1]:
# def get_yahoo_economic_calendar(table_class: str = None) -> pd.DataFrame:
#     """
#     Extrat data from trading economics callendar.

#     Parameters:
#         classe_tabela (str): Class CSS from table to scrap (optional).

#     Returns:
#         pd.DataFrame: DataFrame with table content.
#     """

#     headers = {
#         "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36"
#     }

#     try:
#         response = requests.get("https://tradingeconomics.com/calendar", headers=headers)
#         response.raise_for_status()

#         soup = BeautifulSoup(response.text, 'html.parser')

#         if table_class:
#             tabela = soup.find('table', {'class': table_class})
#         else:
#             tabela = soup.find('table')
        
#         headers = [th.text.strip() for th in tabela.find_all('th')]
#         print(headers)
#         rows = []
#         for row in tabela.find_all('tr')[1:]:
#             cols = [td.text.strip() for td in row.find_all('td')]
#             if cols: 
#                 rows.append(cols)

#         df = pd.DataFrame(rows, columns=headers if headers else None)
        

#     except requests.exceptions.RequestException as e:
#         print(f"Error accessing URL: {e}")
#     except Exception as e:
#         print(f"Error processing data: {e}")

#     return df


In [2]:
# def get_whalewisdom(ticker: str) -> pd.DataFrame:
#     """
#     Extracts data from WhaleWisdom for a given stock ticker.

#     Parameters:
#         ticker (str): Stock ticker symbol.

#     Returns:
#         pd.DataFrame: DataFrame with table content.
#     """

#     headers = {
#         "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36"
#     }

#     try:
#         response = requests.get(f"https://whalewisdom.com/stock/{ticker}", headers=headers)
#         response.raise_for_status()

#         soup = BeautifulSoup(response.text, 'html.parser')
        
#         # Encontrar a div com a classe "v-data-table__wrapper"
#         div_wrapper = soup.find('div', {'class': 'v-window__container'})
       
#         if not div_wrapper:
#             raise ValueError("Div com a classe 'v-window__container' não encontrada.")

#         # Buscar a primeira tabela dentro desta div
#         tabela = div_wrapper.find('table')
#         if not tabela:
#             raise ValueError("Nenhuma tabela encontrada dentro da div especificada.")
        
#         # Coletar os cabeçalhos da tabela ignorando ícones
#         headers = [th.find('span').text.strip() for th in tabela.find_all('th') if th.find('span')]
#         # print(headers)

#         # Encontrando todas as linhas dentro da tabela (tbody > tr)
#         rows = tabela.find("tbody").find_all("tr")
#         print(rows)
#         # Extraindo os dados de cada linha
#         data = []
#         for row in rows:
#             columns = row.find_all("td")

#             # Pegando os valores correspondentes
#             institution_name = columns[0].text.strip()
#             shares_held = columns[2].text.strip()
#             portfolio_value = columns[3].text.strip()
#             percentage_ownership = columns[4].text.strip()
#             last_report_date = columns[-1].text.strip()

#             data.append({
#                 "Instituição": institution_name,
#                 "Ações Detidas": shares_held,
#                 "Valor do Portfólio": portfolio_value,
#                 "Percentual de Participação": percentage_ownership,
#                 "Última Atualização": last_report_date
#             })
#         #print(data)
    
#     except requests.exceptions.RequestException as e:
#         print(f"Erro ao acessar a URL: {e}")
#     except ValueError as e:
#         print(f"Erro na extração de dados: {e}")
#     except Exception as e:
#         print(f"Erro inesperado: {e}")
    
#     return pd.DataFrame()

In [3]:
# df_whale_s = get_whalewisdom(ticker = "ctre")
# df_whale_s

In [4]:
# base_url = 'https://whalewisdom.com/shell/command.html?args=%7B%22command%22:%22filer_lookup%22,%20%22name%22:%22berkshire%22%7D'
# headers = {
#     "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36",
#     "Accept": "application/json"
# }

# response = requests.get(base_url, headers=headers)
# response.raise_for_status()

# response.text

In [5]:
# import hashlib
# import hmac
# import base64
# import json
# import time
# import urllib.parse
# import requests

# class WhaleWisdom:
#     def __init__(self, secret_key, shared_key):
#         self.secret_key = secret_key
#         self.shared_key = shared_key
#         self.timestamp = self.get_timestamp()

#     def get_timestamp(self):
#         """Gera o timestamp no formato UTC"""
#         return time.strftime('%Y-%m-%dT%H:%M:%SZ', time.gmtime())

#     def signature(self, args):
#         """Gera a assinatura HMAC-SHA1 em Base64"""
#         message = f"{args}\n{self.timestamp}".encode('utf-8')
#         hmac_digest = hmac.new(self.secret_key.encode('utf-8'), message, hashlib.sha1).digest()
#         return base64.b64encode(hmac_digest).decode('utf-8')

#     def encode(self, string):
#         """Codifica a string para URL"""
#         return urllib.parse.quote(string, safe='')

#     def endpoint(self, args):
#         """Constrói a URL da API do WhaleWisdom"""
#         encoded_args = self.encode(json.dumps(args))
#         api_sig = self.signature(json.dumps(args))
#         encoded_timestamp = self.encode(self.timestamp)

#         return f"https://whalewisdom.com/shell/command.json?args={encoded_args}&api_shared_key={self.shared_key}&api_sig={api_sig}&timestamp={encoded_timestamp}"

#     def request(self, args):
#         """Faz a requisição à API"""
#         url = self.endpoint(args)
#         response = requests.get(url)

#         if response.status_code == 200:
#             return response.text
#         else:
#             print(f"Erro {response.status_code}: {response.text}")
#             return None


# # **Exemplo de Uso**
# WW_SHARED_KEY = "aTWxWrcILEOGqkAPZtAL"
# WW_SECRET_KEY = "JGlAleUrGy1bWaLoiWwESNIbTHaovA0ocKOnr4MW"

# ww = WhaleWisdom(secret_key=WW_SECRET_KEY, shared_key=WW_SHARED_KEY)

# # Obter as participações (holdings) de um gestor específico
# response = ww.request({
#     "command": "stock_lookup",
#     "symbol": "ctre"
# })

# data = json.loads(response)

# stock_id = data["stocks"][0]["id"]
# name = data["stocks"][0]["name"]
# status = data["stocks"][0]["status"]
# link = data["stocks"][0]["link"]

# stock_id

In [6]:
# response = ww.request(
# {"command":"holdings","stock_ids":[167941],"filer_ids":[373], "limit": 10, "all_quarters":0})
# response


In [7]:
# response

## STOCKS



In [26]:
!pip install --upgrade yfinance

                                              0.0/109.8 kB ? eta -:--:--
     -------------------------------------- 109.8/109.8 kB 6.6 MB/s eta 0:00:00
                                              0.0/64.9 kB ? eta -:--:--
     ---------------------------------------- 64.9/64.9 kB 3.6 MB/s eta 0:00:00
                                              0.0/3.0 MB ? eta -:--:--
     -----                                    0.4/3.0 MB 8.5 MB/s eta 0:00:01
     -------------                            1.0/3.0 MB 10.5 MB/s eta 0:00:01
     -----------------------                  1.8/3.0 MB 12.5 MB/s eta 0:00:01
     -----------------------------------      2.7/3.0 MB 14.3 MB/s eta 0:00:01
     ---------------------------------------- 3.0/3.0 MB 14.9 MB/s eta 0:00:00
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing me


[notice] A new release of pip is available: 23.1 -> 25.0.1
[notice] To update, run: C:\Users\joaom\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [1]:
import yfinance as yf
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
from scipy.signal import argrelextrema
import plotly.graph_objects as go
from tqdm import tqdm
from sklearn.preprocessing import MinMaxScaler


# S&P 500 → ^GSPC
# Dow Jones → ^DJI
# Nasdaq 100 → ^NDX
# Russell 2000 → ^RUT
# DAX (Alemanha) → ^GDAXI
# Definir o ticker da ação
symbol = "AAPL"  # Apple Inc.
# symbol = "^GSPC"  # Apple Inc.
# symbol = "XLK"  # Apple Inc.

# Definir período e intervalo
period = "1y"       # Período de 1 mês
interval = "1d"      # Dados diários

yahoo_symbol_info = yf.Ticker("MSFT").info

In [4]:
def get_data_history(symbol : str, period : str, interval : str, start = '1900-01-01', end = datetime.now(), prepost : bool = True):
        '''
        Data collection from yahoo

        Parameters:
        period : str
            Valid periods: 1d,5d,1mo,3mo,6mo,1y,2y,5y,10y,ytd,max Either Use period parameter or use start and end
        interval : str
            Valid intervals: 1m,2m,5m,15m,30m,60m,90m,1h,1d,5d,1wk,1mo,3mo Intraday data cannot extend last 60 days
        start: str
            Download start date string (YYYY-MM-DD) or _datetime, inclusive. Default is 1900-01-01 E.g. for start="2020-01-01", the first data point will be on "2020-01-01"
        end: str
            Download end date string (YYYY-MM-DD) or _datetime, exclusive. Default is now E.g. for end="2023-01-01", the last data point will be on "2022-12-31"
        prepost : bool
            Include Pre and Post market data in results? Default is False
        '''
        yahoo_data_history = yf.Ticker(symbol).history(period=period, interval=interval, start=start, end=end, prepost=prepost)
        yahoo_data_history.reset_index(inplace=True)

        yahoo_data_history["Date"] = yahoo_data_history["Date"].dt.strftime("%Y-%m-%d %H:%M")

        return yahoo_data_history

data_history = get_data_history("EURCAD=X", period, interval)

In [5]:
def peak_detect(close, order):
    '''
    This function find the peaks of a timeframe    
    '''

    if len(close) < 5:
        return [], [], 0, 0, []

    if isinstance(close, pd.Series):
        close_np = close.values  # Converte apenas os valores se for uma série Pandas
    else:
        close_np = np.array(close)

    max_idx = list(argrelextrema(close_np, comparator=np.greater, order=order)[0])
    min_idx = list(argrelextrema(close_np, comparator=np.less, order=order)[0])

    idx = max_idx + min_idx + [len(close_np)-1]
    idx.sort()

    current_idx = idx[-5:]

    start = min(current_idx)
    end = max(current_idx)

    current_pat = close_np[current_idx]

    return current_idx, current_pat, start, end, idx

In [6]:
current_idx, current_pat, start, end, idx = peak_detect(data_history.Close, order=2)

In [7]:
# import plotly.graph_objects as go

# # Criar gráfico com Plotly
# fig = go.Figure()

# # Linha principal (Série Temporal)
# fig.add_trace(go.Scatter(
#     x=data_history['Date'], 
#     y=data_history['Close'], 
#     mode='lines', 
#     name='Preço de Fechamento'
# ))

# # Destacar Picos e Vales
# fig.add_trace(go.Scatter(
#     x=data_history['Date'].iloc[current_idx], 
#     y=current_pat, 
#     mode='markers', 
#     marker=dict(size=10, color='red'), 
#     name='Picos/Vales'
# ))

# # Configuração do layout
# fig.update_layout(
#     title='Detecção de Picos e Vales - Série Temporal',
#     xaxis_title='Data',
#     yaxis_title='Preço de Fechamento',
#     template='plotly_dark',  # Escolha visual moderna e clara
#     height=600,
#     width=1000
# )

# # Exibir gráfico
# fig.show()

In [8]:
def get_gartley_hp(moves, err_allowed, stop_factor, price_X):
    XA, AB, BC, CD = moves

    AB_range = np.array([.618 - err_allowed, .618 + err_allowed]) * abs(XA)
    BC_range = np.array([.382 - err_allowed, .886 + err_allowed]) * abs(AB)
    CD_range = np.array([1.27 - err_allowed, 1.618 + err_allowed]) * abs(BC)

    targets = {
        "XA": None,
        "AB": None,
        "BC": None,
        "CD": None,
        "TP1": None,
        "TP2": None,
        "TP3": None,
        "STOP": None
    }

    ## === Bullish Gartley === ##
    if XA > 0 and AB < 0 and BC > 0 and CD < 0:
        if AB_range[0] < abs(AB) < AB_range[1] and \
           BC_range[0] < abs(BC) < BC_range[1] and \
           CD_range[0] < abs(CD) < CD_range[1]:

            D = XA + AB + BC + CD
            C = XA + AB + BC
            B = XA + AB

            price_D = price_X + D
            price_C = price_X + C
            price_B = price_X + B

            targets.update({
                "XA": XA,
                "AB": AB,
                "BC": BC,
                "CD": CD,
                "TP1": price_B,
                "TP2": price_C,
                "TP3": price_D + (price_C - price_D) * 1.618,
                "STOP": price_D - abs(CD) * stop_factor
            })

            return 1, targets  # Bullish

    ## === Bearish Gartley === ##
    elif XA < 0 and AB > 0 and BC < 0 and CD > 0:
        if AB_range[0] < abs(AB) < AB_range[1] and \
           BC_range[0] < abs(BC) < BC_range[1] and \
           CD_range[0] < abs(CD) < CD_range[1]:

            D = XA + AB + BC + CD
            C = XA + AB + BC
            B = XA + AB

            price_D = price_X + D
            price_C = price_X + C
            price_B = price_X + B

            targets.update({
                "XA": XA,
                "AB": AB,
                "BC": BC,
                "CD": CD,
                "TP1": price_B,
                "TP2": price_C,
                "TP3": price_D - (price_D - price_C) * 1.618,
                "STOP": price_D + abs(CD) * stop_factor
            })

            return -1, targets  # Bearish

    return np.nan, targets


In [9]:
def get_bat_hp(moves, err_allowed, stop_factor, price_X):
    XA, AB, BC, CD = moves

    AB_range = np.array([.382 - err_allowed, .5 + err_allowed]) * abs(XA)
    BC_range = np.array([.382 - err_allowed, .886 + err_allowed]) * abs(AB)
    CD_range = np.array([1.618 - err_allowed, 2.618 + err_allowed]) * abs(BC)

    targets = {
        "XA": None,
        "AB": None,
        "BC": None,
        "CD": None,
        "TP1": None,
        "TP2": None,
        "TP3": None,
        "STOP": None
    }

    ## === Bullish Bat === ##
    if XA > 0 and AB < 0 and BC > 0 and CD < 0:
        if AB_range[0] < abs(AB) < AB_range[1] and \
           BC_range[0] < abs(BC) < BC_range[1] and \
           CD_range[0] < abs(CD) < CD_range[1]:

            D = XA + AB + BC + CD
            C = XA + AB + BC
            B = XA + AB

            price_D = price_X + D
            price_C = price_X + C
            price_B = price_X + B

            targets.update({
                "XA": XA,
                "AB": AB,
                "BC": BC,
                "CD": CD,
                "TP1": price_B,
                "TP2": price_C,
                "TP3": price_D + (price_C - price_D) * 1.618,
                "STOP": price_D - abs(CD) * stop_factor
            })

            return 1, targets  # Bullish

    ## === Bearish Bat === ##
    elif XA < 0 and AB > 0 and BC < 0 and CD > 0:
        if AB_range[0] < abs(AB) < AB_range[1] and \
           BC_range[0] < abs(BC) < BC_range[1] and \
           CD_range[0] < abs(CD) < CD_range[1]:

            D = XA + AB + BC + CD
            C = XA + AB + BC
            B = XA + AB

            price_D = price_X + D
            price_C = price_X + C
            price_B = price_X + B

            targets.update({
                "XA": XA,
                "AB": AB,
                "BC": BC,
                "CD": CD,
                "TP1": price_B,
                "TP2": price_C,
                "TP3": price_D - (price_D - price_C) * 1.618,
                "STOP": price_D + abs(CD) * stop_factor
            })

            return -1, targets  # Bearish

    return np.nan, targets


In [10]:
def get_butterfly_hp(moves, err_allowed, stop_factor, price_X):
    XA, AB, BC, CD = moves

    AB_range = np.array([.786 - err_allowed, .786 + err_allowed]) * abs(XA)
    BC_range = np.array([.382 - err_allowed, .886 + err_allowed]) * abs(AB)
    CD_range = np.array([1.618 - err_allowed, 2.618 + err_allowed]) * abs(XA)  # ou abs(BC)

    targets = {
        "XA": None,
        "AB": None,
        "BC": None,
        "CD": None,
        "TP1": None,
        "TP2": None,
        "TP3": None,
        "STOP": None,
    }

    ## === Bullish Butterfly === ##
    if XA > 0 and AB < 0 and BC > 0 and CD < 0:
        if AB_range[0] < abs(AB) < AB_range[1] and \
           BC_range[0] < abs(BC) < BC_range[1] and \
           CD_range[0] < abs(CD) < CD_range[1]:

            D = XA + AB + BC + CD
            C = XA + AB + BC
            B = XA + AB

            price_D = price_X + D
            price_C = price_X + C
            price_B = price_X + B

            targets.update({
                "XA": XA,
                "AB": AB,
                "BC": BC,
                "CD": CD,
                "TP1": price_B,
                "TP2": price_C,
                "TP3": price_D + (price_C - price_D) * 1.618,
                "STOP": price_D - abs(CD) * stop_factor
            })

            return 1, targets  # Bullish

    ## === Bearish Butterfly === ##
    elif XA < 0 and AB > 0 and BC < 0 and CD > 0:
        if AB_range[0] < abs(AB) < AB_range[1] and \
           BC_range[0] < abs(BC) < BC_range[1] and \
           CD_range[0] < abs(CD) < CD_range[1]:

            D = XA + AB + BC + CD
            C = XA + AB + BC
            B = XA + AB

            price_D = price_X + D
            price_C = price_X + C
            price_B = price_X + B

            targets.update({
                "XA": XA,
                "AB": AB,
                "BC": BC,
                "CD": CD,
                "TP1": price_B,
                "TP2": price_C,
                "TP3": price_D - (price_D - price_C) * 1.618,
                "STOP": price_D + abs(CD) * stop_factor
            })

            return -1, targets  # Bearish

    return np.nan, targets


In [11]:
def get_crab_hp(moves, err_allowed, stop_factor, price_X):
    XA, AB, BC, CD = moves
                
    AB_range = np.array([.382 - err_allowed, .618 + err_allowed]) * abs(XA)
    BC_range = np.array([.382 - err_allowed, .886 + err_allowed]) * abs(AB)
    CD_range = np.array([2.24 - err_allowed, 3.618 + err_allowed]) * abs(XA)

    targets = {
        "XA": None,
        "AB": None,
        "BC": None,
        "CD": None,
        "TP1": None,
        "TP2": None,
        "TP3": None,
        "STOP": None,
    }

    ## === Bullish Crab === ##
    if XA > 0 and AB < 0 and BC > 0 and CD < 0:
        if AB_range[0] < abs(AB) < AB_range[1] and BC_range[0] < abs(BC) < BC_range[1] and CD_range[0] < abs(CD) < CD_range[1]:

            D = XA + AB + BC + CD
            C = XA + AB + BC
            B = XA + AB

            price_D = price_X + D
            price_C = price_X + C
            price_B = price_X + B

            targets.update({
                "XA": XA,
                "AB": AB,
                "BC": BC,
                "CD": CD,
                "TP1": price_B,
                "TP2": price_C,
                "TP3": price_D + (price_C - price_D) * 1.618,
                "STOP": price_D - abs(CD) * stop_factor
            })

            return 1, targets  # Bullish

    ## === Bearish Crab === ##
    elif XA < 0 and AB > 0 and BC < 0 and CD > 0:
        if AB_range[0] < abs(AB) < AB_range[1] and BC_range[0] < abs(BC) < BC_range[1] and CD_range[0] < abs(CD) < CD_range[1]:

            D = XA + AB + BC + CD
            C = XA + AB + BC
            B = XA + AB

            price_D = price_X + D
            price_C = price_X + C
            price_B = price_X + B

            targets.update({
                "XA": XA,
                "AB": AB,
                "BC": BC,
                "CD": CD,
                "TP1": price_B,
                "TP2": price_C,
                "TP3": price_D - (price_D - price_C) * 1.618,
                "STOP": price_D + abs(CD) * stop_factor
            })

            return -1, targets  # Bearish

    return np.nan, targets


In [12]:
def get_harmonic_pattern(data, err_allowed=0.02, order=5, stop_factor=0.1):
    close = data.Close
    fig = go.Figure()

    # Gráfico principal
    fig.add_trace(go.Scatter(
        x=data.Date,
        y=close,
        mode='lines',
        name='Preço',
        line=dict(color='blue')
    ))

    for i in tqdm(range(5, len(close))):
        current_idx, current_pat, start, end, idx = peak_detect(close.values[:i], order=order)
        if len(current_pat) < 5:
            continue

        XA = current_pat[1] - current_pat[0]
        AB = current_pat[2] - current_pat[1]
        BC = current_pat[3] - current_pat[2]
        CD = current_pat[4] - current_pat[3]
        moves = [XA, AB, BC, CD]

        X_price = current_pat[0]
        
        gart = get_gartley_hp(moves, err_allowed, stop_factor, X_price)
        bat = get_bat_hp(moves, err_allowed, stop_factor, X_price)
        butt = get_butterfly_hp(moves, err_allowed, stop_factor, X_price)
        crab = get_crab_hp(moves, err_allowed, stop_factor, X_price)
        
        patterns = [gart, butt, bat, crab]
        labels = ['gart', 'butt', 'bat', 'crab']
        
        # patterns = [crab]
        # labels = ['crab']
        

        for j, pattern in enumerate(patterns):
            if isinstance(pattern, tuple):
                direction, targets = pattern
                if np.isnan(direction):
                    continue

                label = ('Bearish ' if direction == -1 else 'Bullish ') + labels[j].capitalize()
                color = 'red' if direction == -1 else 'green'

                # Desenhar o padrão
                fig.add_trace(go.Scatter(
                    x=data.Date[current_idx],
                    y=current_pat,
                    mode='markers+lines+text',
                    name=label,
                    marker=dict(size=10, color=color),
                    text=['X', 'A', 'B', 'C', 'D'],
                    textposition="top center"
                ))

                for tp_name in ['TP1', 'TP2', 'TP3']:
                    tp_price = targets[tp_name]
                    if tp_price is None:
                        continue

                    # Target Lines
                    fig.add_trace(go.Scatter(
                        x=[data.Date[current_idx[-1]], data.Date[min(current_idx[-1] + 100, len(data.Date) - 1)]],
                        y=[tp_price, tp_price],
                        mode='lines',
                        name=f'{tp_name} ({label})',
                        line=dict(color='orange'),
                        showlegend=True
                    ))

                # === Stop Loss === 
                stop_price = targets["STOP"]
                if stop_price is not None:
                    fig.add_trace(go.Scatter(
                        x=[data.Date[current_idx[-1]], data.Date[min(current_idx[-1] + 100, len(data.Date) - 1)]],
                        y=[stop_price, stop_price],
                        mode='lines',
                        name=f'STOP ({label})',
                        line=dict(color='white'),
                        showlegend=True
                    ))

            elif pattern == 1 or pattern == -1:
                label = ('Bearish ' if pattern == -1 else 'Bullish ') + labels[j].capitalize()
                color = 'red' if pattern == -1 else 'green'

                fig.add_trace(go.Scatter(
                    x=data.Date[current_idx],
                    y=current_pat,
                    mode='markers+lines+text',
                    name=label,
                    marker=dict(size=10, color=color),
                    text=['X', 'A', 'B', 'C', 'D'],
                    textposition="top center"
                ))

    fig.update_layout(
        title="Detecção de Padrão Harmônico com Targets",
        xaxis_title="Data",
        yaxis_title="Preço de Fecho",
        legend_title="Padrões Detectados",
        template="plotly_dark"
    )

    fig.show()


In [17]:
def backtest_harmonic_patterns(data, err_allowed=0.02, order=5, stop_factor=0.1, future_window=20):
    close = data.Close.values
    results = []

    for i in tqdm(range(5, len(close) - future_window)):
        current_idx, current_pat, start, end, idx = peak_detect(close[:i], order=order)
        if len(current_pat) < 5:
            continue

        XA = current_pat[1] - current_pat[0]
        AB = current_pat[2] - current_pat[1]
        BC = current_pat[3] - current_pat[2]
        CD = current_pat[4] - current_pat[3]
        moves = [XA, AB, BC, CD]
        price_X = current_pat[0]

        patterns = [
            get_gartley_hp(moves, err_allowed, stop_factor, price_X),
            get_butterfly_hp(moves, err_allowed, stop_factor, price_X),
            get_bat_hp(moves, err_allowed, stop_factor, price_X),
            get_crab_hp(moves, err_allowed, stop_factor, price_X)
        ]

        pattern_names = ['Gartley', 'Butterfly', 'Bat', 'Crab']

        valid_patterns = [
            (name, p[0], p[1])
            for name, p in zip(pattern_names, patterns)
            if isinstance(p, tuple) and not np.isnan(p[0])
        ]

        if not valid_patterns:
            continue

        best = min(valid_patterns, key=lambda x: x[2].get("CD_DIFF", float('inf')))
        name, direction, targets = best

        D_index = current_idx[-1]
        D_price = current_pat[-1]
        stop_price = targets.get("STOP")
        future_prices = close[D_index:D_index + future_window]

        hit_tp = None
        stop_hit = False

        for price in future_prices:
            if direction == 1 and price <= stop_price:
                stop_hit = True
                break
            elif direction == -1 and price >= stop_price:
                stop_hit = True
                break

            for tp_name in ['TP1', 'TP2', 'TP3']:
                tp = targets[tp_name]
                if tp is None:
                    continue

                if (direction == 1 and price >= tp) or (direction == -1 and price <= tp):
                    hit_tp = tp_name
                    break
            if hit_tp:
                break

        # ==== Reward/Risk ====
        reward = None
        risk = None
        rr_ratio = None
        weighted_return = None

        if hit_tp:
            reward = abs(targets[hit_tp] - D_price)
        if stop_price:
            risk = abs(stop_price - D_price)
        if reward and risk and risk > 0:
            rr_ratio = reward / risk

        if hit_tp:
            weighted_return = reward
        elif stop_hit:
            weighted_return = -risk

        results.append({
            "pattern": name,
            "direction": direction,
            "hit_tp": hit_tp,
            "stop_hit": stop_hit,
            "D_index": D_index,
            "D_price": D_price,
            "CD_DIFF": targets.get("CD_DIFF"),
            "order": order,
            "err_allowed": err_allowed,
            "reward": reward,
            "risk": risk,
            "rr_ratio": rr_ratio,
            "weighted_return": weighted_return,
            "pattern_idx": current_idx,
            "pattern_idx_dates": data.Date.iloc[current_idx].tolist(),    
            "pattern_prices": current_pat
        })

    return results


In [18]:
results = backtest_harmonic_patterns(data_history, err_allowed=0.02, order=5, stop_factor=0.1, future_window=20)
results

100%|██████████| 5513/5513 [00:02<00:00, 2244.52it/s]


[{'pattern': 'Crab',
  'direction': -1,
  'hit_tp': None,
  'stop_hit': True,
  'D_index': 265,
  'D_price': 1.6223000288009644,
  'CD_DIFF': None,
  'order': 5,
  'err_allowed': 0.02,
  'reward': None,
  'risk': 0.0076600074768067294,
  'rr_ratio': None,
  'weighted_return': -0.0076600074768067294,
  'pattern_idx': [231, 246, 252, 254, 265],
  'pattern_idx_dates': ['2004-10-19 00:00',
   '2004-11-10 00:00',
   '2004-11-18 00:00',
   '2004-11-22 00:00',
   '2004-12-07 00:00'],
  'pattern_prices': array([1.57299995, 1.54040003, 1.56110001, 1.54569995, 1.62230003])},
 {'pattern': 'Crab',
  'direction': -1,
  'hit_tp': None,
  'stop_hit': True,
  'D_index': 266,
  'D_price': 1.6233999729156494,
  'CD_DIFF': None,
  'order': 5,
  'err_allowed': 0.02,
  'reward': None,
  'risk': 0.0077700018882751465,
  'rr_ratio': None,
  'weighted_return': -0.0077700018882751465,
  'pattern_idx': [231, 246, 252, 254, 266],
  'pattern_idx_dates': ['2004-10-19 00:00',
   '2004-11-10 00:00',
   '2004-11-18 0

In [ ]:
def simulate_trading(results, initial_capital=10000, risk_per_trade=0.01):
    import pandas as pd

    df = pd.DataFrame(results)
    df = df.dropna(subset=["risk"]) 

    capital = initial_capital
    equity_curve = []
    trades = []

    for idx, row in df.iterrows():
        risk_amount = capital * risk_per_trade

        if row['risk'] == 0 or pd.isna(row['risk']):
            continue  # evitar divisões por zero

        # Tamanho da posição baseado no risco
        position_size = risk_amount / row['risk']

        if row['hit_tp']:
            gain = position_size * row['reward']
            capital += gain
            trades.append({"result": "win", "pnl": gain})
        elif row['stop_hit']:
            loss = -risk_amount
            capital += loss
            trades.append({"result": "loss", "pnl": loss})
        else:
            # Nenhum TP ou stop atingido → neutro
            trades.append({"result": "neutral", "pnl": 0})

        equity_curve.append(capital)

    equity_df = pd.DataFrame(trades)
    equity_df['capital'] = equity_curve

    return equity_df


In [ ]:
def normalize_decision_rank(resumo):
    resumo_norm = resumo.copy()

    # Garantir que não há NaNs nas colunas-chave antes de normalizar
    cols_to_normalize = [
        "capital_final",
        "expectancy",
        "reward_risk_medio",
        "retorno_ponderado_medio",
        "lucro_por_trade"
    ]

    # Preencher NaNs com o mínimo (tratamento conservador)
    for col in cols_to_normalize:
        if col in resumo_norm.columns:
            min_val = resumo_norm[col].min()
            resumo_norm[col] = resumo_norm[col].fillna(min_val)

    # Normalizar (0 a 1)
    scaler = MinMaxScaler()
    normalized = pd.DataFrame(
        scaler.fit_transform(resumo_norm[cols_to_normalize]),
        columns=[col + "_norm" for col in cols_to_normalize]
    )

    # Juntar colunas normalizadas
    resumo_norm = pd.concat([resumo_norm.reset_index(drop=True), normalized], axis=1)

    # Score ponderado (ajuste pesos conforme a tua estratégia)
    resumo_norm["score"] = (
        resumo_norm["capital_final_norm"] * 0.40 +
        resumo_norm["expectancy_norm"] * 0.25 +
        resumo_norm["reward_risk_medio_norm"] * 0.15 +
        resumo_norm["retorno_ponderado_medio_norm"] * 0.15 +
        resumo_norm["lucro_por_trade_norm"] * 0.05
    )

    # Ranking
    resumo_norm = resumo_norm.sort_values(by="score", ascending=False)
    resumo_norm["ranking"] = range(1, len(resumo_norm) + 1)

    return resumo_norm


In [ ]:
def run_full_backtest(data, future_window=20):
    """
    Testa várias combinações de (order, err_allowed, stop_factor) e avalia performance dos padrões harmônicos.
    
    Return:
        summary_df: DataFrame com estatísticas de acerto e capital final por combinação
    """

    # orders = [2, 3, 4, 5]
    # err_values = [0.01, 0.02, 0.03, 0.04, 0.05, 0.06, 0.07, 0.08, 0.09, 0.1]
    # stop_factors = [0.1, 0.2]
    orders = [5]
    err_values = [0.05, 0.06]
    stop_factors = [0.1, 0.2]
    capex = 10000
    risk_per_trade=0.01

    summary = []

    for order in orders:
        for err in err_values:
            for stop in stop_factors:
                
                # Roda o backtest
                results = backtest_harmonic_patterns(
                    data,
                    err_allowed=err,
                    order=order,
                    stop_factor=stop,
                    future_window=future_window
                )
                df = pd.DataFrame(results)

                total = len(df)
                hits = df['hit_tp'].notna().sum()
                success_rate = hits / total if total > 0 else 0

                # Estatísticas de retorno
                rr_mean = df['rr_ratio'].mean()
                reward_mean = df['reward'].mean()
                risk_mean = df['risk'].mean()
                weighted_return_mean = df['weighted_return'].mean()

                # Simulação de capital
                equity_df = simulate_trading(results, initial_capital=capex)
                final_capital = equity_df['capital'].iloc[-1] if not equity_df.empty else None

                taxa_acerto = round(success_rate * 100, 2)
                reward_medio = round(reward_mean, 4) if reward_mean else None
                risk_medio = round(risk_mean, 4) if risk_mean else None
                reward_risk_medio = round(rr_mean, 4) if rr_mean else None
                retorno_ponderado_medio = round(weighted_return_mean, 4) if weighted_return_mean else None
                capital_final = round(final_capital, 2) if final_capital else None

                expectancy = (
                    (taxa_acerto / 100) * reward_risk_medio - (1 - taxa_acerto / 100)
                ) if reward_risk_medio is not None else None

                lucro_por_trade = (
                    (capital_final - capex) / total if total > 0 else None
                )

                summary.append({
                    "order": order,
                    "err_allowed": err,
                    "stop_factor": stop,
                    "padroes_detectados": total,
                    "targets_atingidos": hits,
                    "taxa_acerto": taxa_acerto,
                    "reward_medio": reward_medio,
                    "risk_medio": risk_medio,
                    "reward_risk_medio": reward_risk_medio,
                    "retorno_ponderado_medio": retorno_ponderado_medio,
                    "expectancy": expectancy,
                    "lucro_por_trade": lucro_por_trade,
                    "capital_final": capital_final
                })

    summary_df = pd.DataFrame(summary)
    summary_df = summary_df.sort_values(by="capital_final", ascending=False)

    summary_df = normalize_decision_rank(summary_df)

    best_err_allowed = summary_df.loc[summary_df['ranking']==1, 'err_allowed'].values[0]
    best_order = summary_df.loc[summary_df['ranking']==1, 'order'].values[0]
    best_stop_factor = summary_df.loc[summary_df['ranking']==1, 'stop_factor'].values[0]

    return summary_df



In [ ]:
resumo = run_full_backtest(data_history)

100%|██████████| 5417/5417 [00:02<00:00, 2181.65it/s]


In [ ]:
resumo

,order,err_allowed,stop_factor,padroes_detectados,targets_atingidos,taxa_acerto,reward_medio,risk_medio,reward_risk_medio,retorno_ponderado_medio,expectancy,lucro_por_trade,capital_final,capital_final_norm,expectancy_norm,reward_risk_medio_norm,retorno_ponderado_medio_norm,lucro_por_trade_norm,score,ranking
0,5,0.05,0.1,31,10,32.26,0.0147,0.0038,4.5862,0.0019,0.802108,86.037419,12667.16,1.000000,1.00000,1.0,1.000000,1.000000,1.000000,1
1,5,0.06,0.1,33,10,30.30,0.0147,0.0038,4.5862,0.0016,0.692619,73.184545,12415.09,0.903544,0.86387,1.0,0.884615,0.847725,0.902464,2
2,5,0.05,0.2,31,10,32.26,0.0147,0.0076,2.2931,-0.0003,0.062354,8.321613,10257.97,0.078111,0.08025,0.0,0.153846,0.079261,0.078347,3
3,5,0.06,0.2,33,10,30.30,0.0147,0.0076,2.2931,-0.0007,-0.002191,1.631515,10053.84,0.000000,0.00000,0.0,0.000000,0.000000,0.000000,4


In [ ]:
best_err_allowed = resumo.loc[resumo['ranking']==1, 'err_allowed'].values[0]
best_order = resumo.loc[resumo['ranking']==1, 'order'].values[0]
best_stop_factor = resumo.loc[resumo['ranking']==1, 'stop_factor'].values[0]

In [106]:
get_harmonic_pattern(data_history, err_allowed=best_err_allowed, order=best_order, stop_factor=best_stop_factor)

100%|██████████| 5532/5532 [00:02<00:00, 2036.07it/s]


In [11]:
# https://medium.com/towards-data-science/pivot-points-calculation-in-python-for-day-trading-659c1e92d323


# ### ZONAS PIVOT! ###

## Data Collection Binance


In [ ]:
# import requests

# url = "https://api.binance.com/api/v3/ticker/price"
# params = {"symbol": "BTCUSDT"}

# response = requests.get(url, params=params)

# if response.status_code == 200:
#     symbol = response.json()["symbol"]
#     price = response.json()["price"]
# else:
#     error = response.status_code

In [ ]:
# from decimal import Decimal
# from datetime import datetime


# def get_crypto_symbol_24h(symbol : str):

#     url = "https://api.binance.com/api/v3/ticker/24hr"
#     params = {"symbol": symbol}

#     response = requests.get(url, params=params)

#     if response.status_code == 200:
#         priceChangePercent = Decimal(response.json()["priceChangePercent"])
#         weightedAvgPrice = Decimal(response.json()["weightedAvgPrice"])
#         prevClosePrice = Decimal(response.json()["prevClosePrice"])
#         priceChange = Decimal(response.json()["priceChange"])
#         lastPrice = Decimal(response.json()["lastPrice"])
#         lastQty = Decimal(response.json()["lastQty"])
#         bidPrice = Decimal(response.json()["bidPrice"])
#         bidQty = Decimal(response.json()["bidQty"])
#         askPrice = Decimal(response.json()["askPrice"])
#         askQty = Decimal(response.json()["askQty"])
#         openPrice = Decimal(response.json()["openPrice"])
#         highPrice = Decimal(response.json()["highPrice"])
#         lowPrice = Decimal(response.json()["lowPrice"])
#         volume = Decimal(response.json()["volume"])
#         quoteVolume = Decimal(response.json()["quoteVolume"])
#         openTime = datetime.fromtimestamp(response.json()["openTime"] / 1000).strftime("%d-%m-%Y %H:%M:%S")
#         closeTime = datetime.fromtimestamp(response.json()["closeTime"] / 1000).strftime("%d-%m-%Y %H:%M:%S")
#         firstId = response.json()["firstId"]
#         lastId = response.json()["lastId"]
#         count = response.json()["count"]
#     else:
#         print(f"Erro: {response.status_code} - {response.text}")

In [ ]:
# url = "https://api.binance.com/api/v3/openOrders"
# params = {"symbol": "BTCUSDT"}

# response = requests.get(url, params=params)
# response.json()
